# load embedding

In [1]:
#kor2vec으로 학습시킨 embedding

from kor2vec import Kor2Vec
kor2vec = Kor2Vec.load("embedding_model")

c:\users\이육샛별\appdata\local\programs\python\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#tokenizing 후 따로 학습시킨 word2vec 모델로 단어셋을 만들었음.

import gensim
from gensim.models import Word2Vec, KeyedVectors
from gensim.models import KeyedVectors

model = gensim.models.Word2Vec.load('ko_new.bin')

#단어셋에 있는 각 단어들이 kor2vec으로 임베딩됨.
words = set(model.wv.vocab)
word_to_vec_map = {}

for w in words:
    word_to_vec_map[w] = kor2vec.embedding(w).detach().numpy()

In [22]:
import torch
#cosine similiarity
cos = torch.nn.CosineSimilarity(dim=1)

male_keyword = torch.Tensor(word_to_vec_map["아빠"])
female_keyword = torch.Tensor(word_to_vec_map["엄마"])
search_keyword = torch.Tensor(word_to_vec_map["부엌"])

output = [cos(male_keyword,search_keyword), cos(female_keyword,search_keyword)]

male_similarity = str(str(output[0]).split('[')[1]).split(']')[0]
female_similarity = str(str(output[1]).split('[')[1]).split(']')[0]

print("아빠 - 부엌" +':', float(male_similarity))
print("엄마 - 부엌" + ':', float(female_similarity))

아빠 - 부엌: 0.0233
엄마 - 부엌: 0.1949


# dataset

In [142]:
deifinitional = [["여성", "남성"],["소녀", "소년"], ["의사","간호사"], ["어머니", "아버지"],["엄마","아빠"], ["아내","남편"],["언니","오빠"]]
equalized_set = [["할머니","할아버지"],["손녀","손자"],["삼촌","이모"],["아내","남편"],["어머니", "아버지"],["엄마","아빠"], ["언니","오빠"]]
gender_specific_words = ["그", "그녀","남성","여성","엄마","아빠","소녀","소년","어머니","아버지"]

In [81]:
from sklearn.decomposition import PCA

def gender_subspace(pairs, word_to_vec_map):
    for a,b in pairs:
        gender_axis = word_to_vec_map[a] - word_to_vec_map[b]
        #center = (word_to_vec_map[a]+word_to_vec_map[b])/2
        #pca = PCA(n_components = 0.99)
        #pca = pca.fit_transform(gender_axis)
    return gender_axis

In [69]:
g = gender_subspace(deifinitional, word_to_vec_map)
g.shape

(1, 128)

In [82]:
g2 = gender_subspace([['엄마','아빠']], word_to_vec_map)
g2.shape

(1, 128)

In [78]:
import numpy as np

def neutralize(wordset, gender_axis, word_to_vec_map):
    for w in wordset:
        e = word_to_vec_map[w]`
        e = e.reshape(128,1)
        
    g = gender_axis
    e_biascomponent = (e.dot(g)/np.sum(g*g)) * g
    
    e_debiased = e - e_biascomponent
    
    e_debiased = torch.Tensor(e_debiased)
    return e_debiased

In [85]:
keyword_list = ['아기','공주','부엌','노동','농사','과학']

e_debiased = neutralize(keyword_list,g2,word_to_vec_map)

In [86]:
for i in range(len(keyword_list)):
    print("cosine similarity between " + keyword_list[i] + " and g, before neutralizing: ",cos(torch.Tensor(word_to_vec_map[keyword_list[i]]), torch.Tensor(g2)))
    print("cosine similarity between " + keyword_list[i] + " and g, after neutralizing: ", cos(torch.Tensor(e_debiased[i]),torch.Tensor(g2)))
    print('-------------------------------------')

cosine similarity between 아기 and g, before neutralizing:  tensor([-0.1731])
cosine similarity between 아기 and g, after neutralizing:  tensor([-0.0068])
-------------------------------------
cosine similarity between 공주 and g, before neutralizing:  tensor([-0.1313])
cosine similarity between 공주 and g, after neutralizing:  tensor([0.0068])
-------------------------------------
cosine similarity between 부엌 and g, before neutralizing:  tensor([0.1576])
cosine similarity between 부엌 and g, after neutralizing:  tensor([0.0068])
-------------------------------------
cosine similarity between 노동 and g, before neutralizing:  tensor([0.0735])
cosine similarity between 노동 and g, after neutralizing:  tensor([-0.0068])
-------------------------------------
cosine similarity between 농사 and g, before neutralizing:  tensor([-0.0620])
cosine similarity between 농사 and g, after neutralizing:  tensor([0.0068])
-------------------------------------
cosine similarity between 과학 and g, before neutralizing:  te

In [148]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    
    ### START CODE HERE ###
    # Step 1: Select word vector representation of "word". Use word_to_vec_map. (≈ 2 lines)
    w1, w2 = pair[0], pair[1]
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]
    
    # Step 2: Compute the mean of e_w1 and e_w2 (≈ 1 line)
    mu = (e_w1 + e_w2) / 2
    bias_axis = bias_axis.reshape(128,1)
    # Step 3: Compute the projections of mu over the bias axis and the orthogonal axis (≈ 2 lines)
    mu_B = mu.dot(bias_axis)/np.sum(bias_axis*bias_axis) * bias_axis
    mu_orth = mu - mu_B
 
    # Step 4: Use equations (7) and (8) to compute e_w1B and e_w2B (≈2 lines)
    e_w1B = e_w1.dot(bias_axis)/np.sum(bias_axis*bias_axis) * bias_axis
    e_w2B = e_w2.dot(bias_axis)/np.sum(bias_axis*bias_axis) * bias_axis
        
    # Step 5: Adjust the Bias part of e_w1B and e_w2B using the formulas (9) and (10) given above (≈2 lines)
    corrected_e_w1B = np.sqrt(np.abs(1-np.sum(mu_orth*mu_orth))) * (e_w1B - mu_B) / np.sqrt(np.sum((e_w1 - mu_orth - mu_B)**2))
    corrected_e_w2B = np.sqrt(np.abs(1-np.sum(mu_orth*mu_orth))) * (e_w2B - mu_B) / np.sqrt(np.sum((e_w2 - mu_orth - mu_B)**2))
 
    # Step 6: Debias by equalizing e1 and e2 to the sum of their corrected projections (≈2 lines)
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth
                                                                
    ### END CODE HERE ###

    return e1,e2

In [110]:
e = equalize(equalized_set, g, word_to_vec_map)

In [117]:
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cos(torch.Tensor(word_to_vec_map[equalized_set[0][0]]), torch.Tensor(g)))

cosine_similarity(word_to_vec_map["man"], gender) =  tensor([0.1570])


In [160]:
for i in range(len(equalized_set)):
    print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cos(torch.Tensor(word_to_vec_map[equalized_set[i][0]]),torch.Tensor(g)))
    print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cos(torch.Tensor(word_to_vec_map[equalized_set[i][1]]),torch.Tensor(g)))

cosine_similarity(word_to_vec_map["man"], gender) =  tensor([0.1570])
cosine_similarity(word_to_vec_map["woman"], gender) =  tensor([0.0948])
cosine_similarity(word_to_vec_map["man"], gender) =  tensor([-0.0422])
cosine_similarity(word_to_vec_map["woman"], gender) =  tensor([0.0541])
cosine_similarity(word_to_vec_map["man"], gender) =  tensor([0.0467])
cosine_similarity(word_to_vec_map["woman"], gender) =  tensor([-0.1166])
cosine_similarity(word_to_vec_map["man"], gender) =  tensor([-0.1172])
cosine_similarity(word_to_vec_map["woman"], gender) =  tensor([-0.0411])
cosine_similarity(word_to_vec_map["man"], gender) =  tensor([0.0755])
cosine_similarity(word_to_vec_map["woman"], gender) =  tensor([0.1032])
cosine_similarity(word_to_vec_map["man"], gender) =  tensor([-0.2565])
cosine_similarity(word_to_vec_map["woman"], gender) =  tensor([-0.6325])
cosine_similarity(word_to_vec_map["man"], gender) =  tensor([0.5520])
cosine_similarity(word_to_vec_map["woman"], gender) =  tensor([-0.7203])

In [152]:
print("cosine similarities after equalizing:")
for i in range(len(equalized_set)):
    e1, e2 = equalize(equalized_set[i], g, word_to_vec_map)
    print("cosine_similarity(e1, gender) = ", cos(torch.Tensor(e1), torch.Tensor(g)))
    print("cosine_similarity(e2, gender) = ", cos(torch.Tensor(e2), torch.Tensor(g)))

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  tensor([0.1486, 0.1486, 0.1486, 0.1486, 0.1486, 0.1486, 0.1486, 0.1486, 0.1486,
        0.1482, 0.1475, 0.1486, 0.1484, 0.1486, 0.1480, 0.1486, 0.1473, 0.1486,
        0.1485, 0.1486, 0.1485, 0.1486, 0.1364, 0.1486, 0.1486, 0.1484, 0.1483,
        0.1475, 0.1486, 0.1417, 0.1466, 0.1486, 0.1461, 0.1486, 0.1353, 0.1468,
        0.1483, 0.1467, 0.1486, 0.1486, 0.1474, 0.1484, 0.1483, 0.1486, 0.1486,
        0.1486, 0.1483, 0.1485, 0.1485, 0.1485, 0.1485, 0.1472, 0.1457, 0.1485,
        0.1485, 0.1474, 0.1476, 0.1485, 0.1481, 0.1486, 0.1485, 0.1486, 0.1486,
        0.1422, 0.1432, 0.1486, 0.1437, 0.1462, 0.1480, 0.1485, 0.1472, 0.1485,
        0.1485, 0.1471, 0.1485, 0.1484, 0.1483, 0.1481, 0.1418, 0.1464, 0.1484,
        0.1485, 0.1473, 0.1472, 0.1478, 0.1473, 0.1485, 0.1486, 0.1484, 0.1483,
        0.1427, 0.1486, 0.1444, 0.1470, 0.1477, 0.1465, 0.1485, 0.1445, 0.1462,
        0.1481, 0.1449, 0.1464, 0.1465, 0.1480, 0

In [161]:
e

(array([[-0.14632879, -0.24495801, -0.17242455, ..., -0.11753263,
          0.19046134, -0.16749611],
        [-0.14632879, -0.24495801, -0.17242455, ..., -0.11753263,
          0.19046134, -0.16749611],
        [-0.14632879, -0.24495801, -0.17242455, ..., -0.11753263,
          0.19046134, -0.16749611],
        ...,
        [-0.39549857, -0.4941278 , -0.42159435, ..., -0.36670244,
         -0.05870846, -0.4166659 ],
        [ 0.12313469,  0.02450547,  0.09703892, ...,  0.15193085,
          0.45992482,  0.10196736],
        [-0.77326775, -0.8718969 , -0.7993635 , ..., -0.74447155,
         -0.4364776 , -0.794435  ]], dtype=float32),
 array([[-1.46328792e-01, -2.44958013e-01, -1.72424555e-01, ...,
         -1.17532626e-01,  1.90461338e-01, -1.67496115e-01],
        [-1.46328792e-01, -2.44958013e-01, -1.72424555e-01, ...,
         -1.17532626e-01,  1.90461338e-01, -1.67496115e-01],
        [-1.46328792e-01, -2.44958013e-01, -1.72424555e-01, ...,
         -1.17532626e-01,  1.90461338e-01